In [55]:
import sys
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER/')
import os
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
from sklearn.metrics import precision_recall_fscore_support as score
from numpy import newaxis
import sklearn
import subprocess
import fastText

In [56]:
def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]


def createMatrices(sentences, label2Idx, case2Idx,char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}

        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []
        caseIndices = []
        charIndices = []
        labelIndices = []
        
        for word,char,label in sentence:  
            charIdx = []
            for x in char:
                if x in char2Idx.keys():
                    charIdx.append(char2Idx[x])
                else:
                    charIdx.append(char2Idx['UNKNOWN'])
            #Get the label and map to int            
            wordIndices.append(word)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices, caseIndices, charIndices, labelIndices]) 
        
    return dataset

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences



In [57]:
epochs = 80
trainable = True

In [58]:
from keras_contrib.layers import CRF


In [59]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [6]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [60]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [8]:

# preproecessing data from Conll
def get_sentences(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [61]:
trainSentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-train.tsv')
devSentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-dev.tsv')
testSentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-test.tsv')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


23999
2199
5099


In [ ]:
# conll
train_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.train')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testa')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testb')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


In [62]:
print(trainSentences[100])

[['Der', 'O'], ['Meteoritensucher', 'O'], ['hatte', 'O'], ['im', 'O'], ['Frühjahr', 'O'], ['auf', 'O'], ['der', 'O'], ['dänischen', 'B-LOCderiv'], ['Insel', 'O'], ['Lolland', 'B-LOC'], ['30', 'O'], ['Gramm', 'O'], ['eines', 'O'], ['Meteoriten', 'O'], ['aufgespürt', 'O'], [',', 'O'], ['der', 'O'], ['Mitte', 'O'], ['Januar', 'O'], ['über', 'O'], ['Nordeuropa', 'B-LOC'], ['beobachtet', 'O'], ['worden', 'O'], ['war', 'O'], ['.', 'O']]


In [63]:
labelSet = set()
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)

In [65]:
print(len(labelSet))

25


In [15]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [16]:
print(label2Idx)

{'I-ORGderiv': 10, 'B-OTHpart': 11, 'B-LOCderiv': 0, 'B-OTH': 12, 'B-LOC': 1, 'I-LOCderiv': 14, 'I-PERpart': 15, 'B-ORG': 3, 'B-ORGpart': 2, 'B-ORGderiv': 4, 'I-ORG': 16, 'I-OTHderiv': 17, 'B-PERderiv': 5, 'I-OTH': 18, 'I-LOC': 6, 'I-ORGpart': 19, 'O': 21, 'B-PER': 20, 'I-LOCpart': 7, 'B-LOCpart': 22, 'I-PERderiv': 8, 'I-OTHpart': 23, 'I-PER': 24, 'B-PERpart': 13, 'B-OTHderiv': 9}


In [17]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [18]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'numeric': 0, 'allUpper': 2, 'mainly_numeric': 5, 'other': 4, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'allLower': 1, 'initialUpper': 3}


In [ ]:
print(type(trainSentences))

In [19]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [20]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
char2Idx['UNKNOWN'] = len(char2Idx)
print(char2Idx)

{'ї': 225, '»': 0, 'ñ': 1, 'オ': 2, 'É': 3, 'û': 110, 'ά': 4, 'Е': 11, 'ã': 6, ':': 8, 'λ': 10, '€': 57, '守': 278, 'п': 112, '\x9a': 13, 'ö': 230, 'ž': 14, 'ـ': 15, '<': 16, '鶴': 285, '루': 19, '¹': 18, 'ß': 20, '造': 21, 't': 22, 'æ': 24, 'к': 25, 'ο': 26, 'б': 28, 'β': 171, '\x96': 29, '0': 31, 'İ': 35, 'ğ': 34, 'Þ': 313, 'ê': 37, 'F': 39, 'D': 281, '⋅': 40, 'À': 7, '”': 282, 'ε': 42, '\x95': 5, '…': 44, 'Å': 46, 'й': 47, 'B': 297, 'ю': 176, 'н': 49, 'т': 50, 'ż': 51, '›': 52, 'ǒ': 54, '\xad': 55, 'о': 56, 'L': 58, 'в': 60, 'å': 61, 'Л': 62, '▪': 63, 'ł': 65, 'ĩ': 66, '¸': 67, 'ḳ': 69, 'ı': 70, 'σ': 74, 'O': 72, 'K': 73, '8': 75, 'υ': 76, 'ن': 68, '%': 77, 'Ш': 259, 'ő': 78, 'ē': 80, 'τ': 81, 'I': 82, '학': 121, 'Î': 27, 'ň': 252, 'ņ': 83, '/': 85, 'ラ': 87, '(': 315, '≘': 88, '@': 89, 'H': 90, '貴': 91, 'm': 229, '4': 122, 'ę': 92, 'W': 93, 'έ': 100, 'U': 95, 'ب': 103, '~': 183, 'ô': 99, 'П': 101, 'A': 239, 'ψ': 106, 's': 104, 'ς': 105, 'В': 71, 'Ş': 43, '→': 184, 'x': 109, 'ā': 293, '¤':

In [21]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [22]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [36]:
def train_generator(dataset: 'list of training/dev sentences', batch_size:'specify the batch size'):
    global line_number
    while True:
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
        output_labels = []
        total_lines = len(dataset)
        for i in range(batch_size):            
            index = line_number%total_lines
            line_number+=1
            sentence = dataset[index]
            print(sentence)
            temp_casing = []
            temp_char=[]
            temp_word=[]
            temp_output=[]
            for word in sentence:
                word, label = word
                casing =getCasing(word, case2Idx)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    if char in char2Idx.keys():
                        temp_char2.append(char2Idx[char])
                    else:
                        temp_char2.append(char2Idx['UNKNOWN']) # To incorporate the words which are not in the vocab
                temp_char2 = np.array(temp_char2)
                temp_char.append(temp_char2)
                word_vector = ft.get_word_vector(word.lower())
                temp_word.append(word_vector)
                temp_output.append(label2Idx[label])
            temp_char = pad_sequences(temp_char, 52)
            word_embeddings.append(temp_word)
            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            output_labels.append(temp_output)
            output_labels = np.array(output_labels)
            output_labels = output_labels[...,newaxis]
            print(np.array(word_embeddings).shape)
            print(np.array(case_embeddings).shape)
            print(np.array(char_embeddings).shape)
            print(output_labels.shape)
            print("******************\n\n")
        yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], output_labels)
            
    

In [44]:
line_number=0
for inp, out in train_generator(trainSentences,1):
    word, case, char = inp
    print(word.shape)
    print(case.shape)
    print(char.shape)
    print(out.shape)
    print("********")

[['Firmengründer', 'O'], ['Wolf', 'B-PER'], ['Peter', 'I-PER'], ['Bree', 'I-PER'], ['arbeitete', 'O'], ['Anfang', 'O'], ['der', 'O'], ['siebziger', 'O'], ['Jahre', 'O'], ['als', 'O'], ['Möbelvertreter', 'O'], [',', 'O'], ['als', 'O'], ['er', 'O'], ['einen', 'O'], ['fliegenden', 'O'], ['Händler', 'O'], ['aus', 'O'], ['dem', 'O'], ['Libanon', 'B-LOC'], ['traf', 'O'], ['.', 'O']]
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22, 1)
******************


(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22, 1)
********
[['Ob', 'O'], ['sie', 'O'], ['dabei', 'O'], ['nach', 'O'], ['dem', 'O'], ['Runden', 'O'], ['Tisch', 'O'], ['am', 'O'], ['23.', 'O'], ['April', 'O'], ['in', 'O'], ['Berlin', 'B-LOC'], ['durch', 'O'], ['ein', 'O'], ['pädagogisches', 'O'], ['Konzept', 'O'], ['unterstützt', 'O'], ['wird', 'O'], [',', 'O'], ['ist', 'O'], ['allerdings', 'O'], ['zu', 'O'], ['bezweifeln', 'O'], ['.', 'O']]
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24, 1)
******************


(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24, 1)
********

(1, 20, 300)
(1, 20)
(1, 20, 52)
(1, 20, 1)
******************


(1, 20, 300)
(1, 20)
(1, 20, 52)
(1, 20, 1)
********
[['Neben', 'O'], ['einem', 'O'], ['4-in-1', 'O'], ['Kartenleser', 'O'], ['und', 'O'], ['Bluetooth', 'O'], ['2.0', 'O'], ['hat', 'O'], ['Medion', 'B-ORG'], ['einen', 'O'], ['8-fach', 'O'], ['DVD-', 'O'], ['und', 'O'], ['CD-Brenner', 'O'], ['eingebaut', 'O'], [',', 'O'], ['der', 'O'], ['auch', 'O'], ['Double-Layer-Medien', 'O'], ['unterstützt', 'O'], ['.', 'O']]
(1, 21, 300)
(1, 21)
(1, 21, 52)
(1, 21, 1)
******************


(1, 21, 300)
(1, 21)
(1, 21, 52)
(1, 21, 1)
********
[['Im', 'O'], ['Mittelalter', 'O'], ['fand', 'O'], ['die', 'O'], ['Schlacht', 'B-OTH'], ['von', 'I-OTH'], ['Worringen', 'I-OTH'], ['am', 'O'], ['5.', 'O'], ['Juni', 'O'], ['1288', 'O'], ['in', 'O'], ['der', 'O'], ['"', 'O'], ['Fühlinger', 'B-LOC'], ['Heide', 'I-LOC'], ['"', 'O'], ['statt', 'O'], ['.', 'O']]
(1, 19, 300)
(1, 19)
(1, 19, 52)
(1, 19, 1)
******************


(1, 19, 300)
(1, 19)
(1, 19

(1, 8, 300)
(1, 8)
(1, 8, 52)
(1, 8, 1)
******************


(1, 8, 300)
(1, 8)
(1, 8, 52)
(1, 8, 1)
********
[['Weitere', 'O'], ['Karriere', 'O'], ['und', 'O'], ['Tod', 'O'], ['Nach', 'O'], ['dem', 'O'], ['Ausscheiden', 'O'], ['aus', 'O'], ['dem', 'O'], ['Amt', 'O'], ['des', 'O'], ['Gouverneurs', 'O'], ['im', 'O'], ['Jahr', 'O'], ['1903', 'O'], ['wurde', 'O'], ['McMillin', 'B-PER'], ['in', 'O'], ['Nashville', 'B-LOC'], ['in', 'O'], ['der', 'O'], ['Versicherungsbranche', 'O'], ['tätig', 'O'], ['.', 'O']]
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24, 1)
******************


(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24, 1)
********
[['Am', 'O'], ['7.', 'O'], ['Februar', 'O'], ['1528', 'O'], ['erliess', 'O'], ['Bern', 'B-LOC'], ['das', 'O'], ['Reformationsmandat', 'O'], ['.', 'O']]
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9, 1)
******************


(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9, 1)
********
[['Technik', 'O'], ['Für', 'O'], ['damalige', 'O'], ['Verhältnisse', 'O'], ['war', 'O'], ['das', 'O'], ['Spiel

(1, 15, 300)
(1, 15)
(1, 15, 52)
(1, 15, 1)
******************


(1, 15, 300)
(1, 15)
(1, 15, 52)
(1, 15, 1)
********
[['Zur', 'O'], ['Verstärkung', 'O'], ['der', 'O'], ['Kammern', 'O'], ['verwenden', 'O'], ['Maulwurfsgrillen', 'O'], ['Teile', 'O'], ['von', 'O'], ['toten', 'O'], ['Pflanzen', 'O'], [',', 'O'], ['die', 'O'], ['sie', 'O'], ['mit', 'O'], ['ihrem', 'O'], ['Panzer', 'O'], ['an', 'O'], ['die', 'O'], ['feuchten', 'O'], ['Erdwände', 'O'], ['drücken', 'O'], ['.', 'O']]
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22, 1)
******************


(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22, 1)
********
[['Im', 'O'], ['Halbfinale', 'O'], ['der', 'O'], ['Abstiegsrunde', 'O'], ['treten', 'O'], ['die', 'O'], ['Verlierer', 'O'], ['aus', 'O'], ['den', 'O'], ['Viertelfinals', 'O'], ['gegeneinander', 'O'], ['an', 'O'], ['.', 'O']]
(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13, 1)
******************


(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13, 1)
********
[['Ihre', 'O'], ['Geschichte', 'O'], ['setzt', 'O'], ['ein

(1, 27, 300)
(1, 27)
(1, 27, 52)
(1, 27, 1)
******************


(1, 27, 300)
(1, 27)
(1, 27, 52)
(1, 27, 1)
********
[['Seinen', 'O'], ['Sohn', 'O'], ['Arnold', 'B-PER'], ['will', 'O'], ['der', 'O'], ['Alte', 'O'], ['noch', 'O'], ['einmal', 'O'], ['sehen', 'O'], ['.', 'O']]
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10, 1)
******************


(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10, 1)
********
[['Mein', 'O'], ['Vater', 'O'], ['war', 'O'], ['früher', 'O'], ['als', 'O'], ['rücksichtsloser', 'O'], ['Frühaufsteher', 'O'], ['in', 'O'], ['der', 'O'], ['gesamten', 'O'], ['Nachbarschaft', 'O'], ['bekannt', 'O'], ['.', 'O']]
(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13, 1)
******************


(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13, 1)
********
[['Im', 'O'], ['Laufe', 'O'], ['der', 'O'], ['europäischen', 'B-LOCderiv'], ['Musikgeschichte', 'O'], ['wurden', 'O'], ['bestimmte', 'O'], ['Intervalle', 'O'], ['in', 'O'], ['polyphoner', 'O'], ['Musik', 'O'], ['also', 'O'], ['offenbar', 'O'], ['zunächst', 'O']

...

(1, 26, 300)
(1, 26)
(1, 26, 52)
(1, 26, 1)
******************


(1, 26, 300)
(1, 26)
(1, 26, 52)
(1, 26, 1)
********
[['Dabei', 'O'], ['sind', 'O'], ['Bildschirm', 'O'], ['und', 'O'], ['Tastatur', 'O'], ['jeweils', 'O'], ['auf', 'O'], ['einer', 'O'], ['Seite', 'O'], ['.', 'O']]
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10, 1)
******************


(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10, 1)
********
[['Am', 'O'], ['6.', 'O'], ['Dezember', 'O'], ['1861', 'O'], ['kamen', 'O'], ['die', 'O'], ['Verdis', 'B-PER'], ['in', 'O'], ['Petersburg', 'B-LOC'], ['an', 'O'], [',', 'O'], ['um', 'O'], ['die', 'O'], ['Uraufführung', 'O'], ['vorzubereiten', 'O'], ['.', 'O']]
(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16, 1)
******************


(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16, 1)
********
[['Die', 'O'], ['vier', 'O'], ['Kronblätter', 'O'], ['sind', 'O'], ['frei', 'O'], ['.', 'O']]
(1, 6, 300)
(1, 6)
(1, 6, 52)
(1, 6, 1)
******************


(1, 6, 300)
(1, 6)
(1, 6, 52)
(1, 6, 1)
********
[['Damit', 'O'], ['

(1, 20, 300)
(1, 20)
(1, 20, 52)
(1, 20, 1)
******************


(1, 20, 300)
(1, 20)
(1, 20, 52)
(1, 20, 1)
********
[['In', 'O'], ['den', 'O'], ['vergangenen', 'O'], ['Wochen', 'O'], ['hatten', 'O'], ['Priester', 'O'], ['und', 'O'], ['Laienvertreter', 'O'], ['immer', 'O'], ['wieder', 'O'], ['verlangt', 'O'], [',', 'O'], ['der', 'O'], ['abgelöste', 'O'], ['Oberhirte', 'O'], ['solle', 'O'], ['einen', 'O'], ['Wohnsitz', 'O'], ['außerhalb', 'O'], ['des', 'O'], ['Bistums', 'O'], ['nehmen', 'O'], [',', 'O'], ['um', 'O'], ['den', 'O'], ['Prozess', 'O'], ['der', 'O'], ['innerkirchlichen', 'O'], ['Verständigung', 'O'], ['nicht', 'O'], ['zu', 'O'], ['stören', 'O'], ['.', 'O']]
(1, 33, 300)
(1, 33)
(1, 33, 52)
(1, 33, 1)
******************


(1, 33, 300)
(1, 33)
(1, 33, 52)
(1, 33, 1)
********
[['Ihm', 'O'], ['wird', 'O'], ['Hochverrat', 'O'], ['vorgeworfen', 'O'], ['.', 'O']]
(1, 5, 300)
(1, 5)
(1, 5, 52)
(1, 5, 1)
******************


(1, 5, 300)
(1, 5)
(1, 5, 52)
(1, 5, 1)
********
[['Scribn

(1, 17, 300)
(1, 17)
(1, 17, 52)
(1, 17, 1)
******************


(1, 17, 300)
(1, 17)
(1, 17, 52)
(1, 17, 1)
********
[['Dass', 'O'], ['mein', 'O'], ['Englisch', 'B-OTH'], ['gut', 'O'], ['ist', 'O'], [',', 'O'], ['hilft', 'O'], ['natürlich', 'O'], [',', 'O'], ['denn', 'O'], ['Kunden', 'O'], ['kommen', 'O'], ['nicht', 'O'], ['von', 'O'], ['allein', 'O'], ['.', 'O']]
(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16, 1)
******************


(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16, 1)
********
[['Wenn', 'O'], ['ich', 'O'], ['das', 'O'], ['Gefühl', 'O'], ['hätte', 'O'], [',', 'O'], ['mich', 'O'], ['in', 'O'], ['Gefahr', 'O'], ['zu', 'O'], ['bringen', 'O'], [',', 'O'], ['würde', 'O'], ['ich', 'O'], ['nicht', 'O'], ['so', 'O'], ['laufen', 'O'], ['.', 'O']]
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18, 1)
******************


(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18, 1)
********
[['15', 'O'], ['%', 'O'], ['derjenigen', 'O'], ['sind', 'O'], ['unter', 'O'], ['18', 'O'], ['Jahre', 'O'], ['und', 'O'], ['9,6', 'O

(1, 28, 300)
(1, 28)
(1, 28, 52)
(1, 28, 1)
********
[['Fachlich', 'O'], ['exakt', 'O'], ['wird', 'O'], ['das', 'O'], ['Wachkoma', 'O'], ['als', 'O'], ['persistierender', 'O'], ['vegetativer', 'O'], ['Status', 'O'], ['(', 'O'], ['PVS', 'O'], [')', 'O'], ['bezeichnet', 'O'], ['.', 'O']]
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14, 1)
******************


(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14, 1)
********
[['Der', 'O'], ['unerwartet', 'O'], ['schwache', 'O'], ['Umsatz', 'O'], ['geht', 'O'], ['vor', 'O'], ['allem', 'O'], ['auf', 'O'], ['das', 'O'], ['Geschäft', 'O'], ['mit', 'O'], ['GPUs', 'O'], ['im', 'O'], ['Consumerbereich', 'O'], ['zurück', 'O'], ['und', 'O'], ['sei', 'O'], ['auf', 'O'], ['höhere', 'O'], ['Speicherpreise', 'O'], ['und', 'O'], ['eine', 'O'], ['wirtschaftliche', 'O'], ['Schwäche', 'O'], ['in', 'O'], ['Europa', 'B-LOC'], ['und', 'O'], ['China', 'B-LOC'], ['zurückzuführen', 'O'], [',', 'O'], ['so', 'O'], ['Nvidia', 'B-ORG'], ['.', 'O']]


KeyboardInterrupt: 

In [24]:
idx2Label = {v: k for k, v in label2Idx.items()}


In [25]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [53]:
words_input = Input(shape=(None,300),dtype='float32',name='words_input')
# words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5, name='dropout1')(embed_char_out)

# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
# output = concatenate([words, casing, char])

kernel_sizes = (3, 4, 5)
conv_blocks = []
for sz in kernel_sizes[:1]:
    conv = TimeDistributed(Conv1D(
                         kernel_size=sz,
                         filters=32,
                         padding="same",
                         activation="relu",
                         strides=1))(embed_char_out)
    conv = TimeDistributed(MaxPooling1D(52))(conv)
    conv = TimeDistributed(Flatten())(conv)
#     conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
output = concatenate([words_input, casing, conv_blocks[0]])

output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
# model.compile(loss=crf.loss_function, optimizer=Adam(lr = 0.002), metrics=[crf.accuracy])
model.compile(loss='categorical_crossentropy', optimizer='nadam', )
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10528       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_31 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
time_distr

In [54]:
line_number=0
model.fit_generator(train_generator(trainSentences, 1), epochs=1, steps_per_epoch= 1,  )

[['Firmengründer', 'O'], ['Wolf', 'B-PER'], ['Peter', 'I-PER'], ['Bree', 'I-PER'], ['arbeitete', 'O'], ['Anfang', 'O'], ['der', 'O'], ['siebziger', 'O'], ['Jahre', 'O'], ['als', 'O'], ['Möbelvertreter', 'O'], [',', 'O'], ['als', 'O'], ['er', 'O'], ['einen', 'O'], ['fliegenden', 'O'], ['Händler', 'O'], ['aus', 'O'], ['dem', 'O'], ['Libanon', 'B-LOC'], ['traf', 'O'], ['.', 'O']]
Epoch 1/1
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22, 1)
******************


[['Ob', 'O'], ['sie', 'O'], ['dabei', 'O'], ['nach', 'O'], ['dem', 'O'], ['Runden', 'O'], ['Tisch', 'O'], ['am', 'O'], ['23.', 'O'], ['April', 'O'], ['in', 'O'], ['Berlin', 'B-LOC'], ['durch', 'O'], ['ein', 'O'], ['pädagogisches', 'O'], ['Konzept', 'O'], ['unterstützt', 'O'], ['wird', 'O'], [',', 'O'], ['ist', 'O'], ['allerdings', 'O'], ['zu', 'O'], ['bezweifeln', 'O'], ['.', 'O']]
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24, 1)
******************


[['Bayern', 'B-ORG'], ['München', 'I-ORG'], ['ist', 'O'], ['wieder', 'O'], ['alleiniger', 'O'],

ValueError: Error when checking target: expected time_distributed_34 to have shape (None, 25) but got array with shape (22, 1)

In [ ]:
for x in range(15,20):
#     print(i)
    maxf1 = 0
    for epoch in range(epochs):    
        print("Epoch %d/%d"%(epoch, epochs))
        a = Progbar(len(train_batch_len))
        for i, batch in enumerate(iterate_minibatches(train_batch, train_batch_len)):
            labels, tokens, casing, char = batch
            labels = labels.tolist()
    #         print(labels)
            for sentence in labels:
                for i,label in enumerate(sentence):
                    temp = [0]*len(label2Idx)
                    value = label[0]
                    temp[value]=1
                    sentence[i] = temp
            labels = np.array(labels)
    #         print(labels)
            model.train_on_batch([tokens, casing, char], labels)
            a.update(i)
        if epoch >= 50:
            predLabels, correctLabels = tag_dataset(dev_batch)        
#             pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
            predLabels = np.concatenate(predLabels).ravel()
            correctLabels = np.concatenate(correctLabels).ravel()
            f1_dev = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
#             print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))
            print(f1_dev)

            if f1_dev > maxf1:
                model.save('german_ner_conll.h5')
                maxf1 = f1_dev
    #         predLabels, correctLabels = tag_dataset(dev_batch)        
        #     with open("dropout=0.75.txt",'w') as f:
    #         pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
        #     with open('dropout=0.75.txt') as f:
        #         x = str(epoch)+ " "+f1_dev
        #         f.write(x)
        #         f.write('\n')
    #         print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))


    id2word = {v: k for k, v in word2Idx.items()}
    print(len(correctLabels))        

    model.load_weights('german_ner_conll.h5')
    predLabels, correctLabels = tag_dataset(test_batch)        
    # pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
    # print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

    id = 1
    file_name='test_conll'+str(x)+'.txt'
    input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
    f = open(input_file_name,'wb')
    for sentence_number, batch in enumerate(test_batch):
        for word_number, wordid in enumerate(batch[0]):
            word = id2word[wordid]
            true_label = correctLabels[sentence_number][word_number]
            true_label = idx2Label[true_label]
            pred_label = predLabels[sentence_number][word_number]
            pred_label = idx2Label[pred_label]

#             string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
            string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
            string = string.encode()
            f.write(string)
            id+=1

    # command = "perl nereval.perl"
    # directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
    directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
    command = "perl conlleval"
    
    output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_conll'+str(x)+'.txt'
    output_file=open(output_file_name,'w')
    input_file = open(input_file_name)
    import subprocess
    process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_conll)
    out, err = process.communicate()
#     print(out)
    print('******************************************************************************')

In [ ]:
import os
os.getcwd()